In [ ]:
#load_dataset.py
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from eunjeon import Mecab
from glob import glob

# Mecab 형태소 분석기 초기화
mecab = Mecab()

# 데이터셋 폴더 경로
DATASET_DIR = "./data_set"

# JSON 파일 로드
def load_json_files(dataset_dir):
    dataset = []
    json_files = glob(os.path.join(dataset_dir, "*.json"))

    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            dataset.extend(data["dataset"])
    
    return dataset

# 어휘 사전 만들기 (word2idx)
def build_vocab(dataset):
    vocab = set()
    for item in dataset:
        vocab.update(item["tokens"])

    word2idx = {word: idx + 1 for idx, word in enumerate(sorted(vocab))}
    word2idx["<PAD>"] = 0  # 패딩 추가
    return word2idx

# 데이터셋 정의
class KoreanTextDataset(Dataset):
    def __init__(self, dataset, word2idx, max_len=20):
        self.dataset = dataset
        self.word2idx = word2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item["text"]
        tokens = item["tokens"]

        # 토큰을 인덱스로 변환
        indexed_tokens = [self.word2idx.get(tok, 0) for tok in tokens]

        # 패딩 추가
        if len(indexed_tokens) < self.max_len:
            indexed_tokens += [0] * (self.max_len - len(indexed_tokens))
        else:
            indexed_tokens = indexed_tokens[:self.max_len]

        return torch.tensor(indexed_tokens), text

# 전체 데이터 로드
dataset = load_json_files(DATASET_DIR)
word2idx = build_vocab(dataset)
train_dataset = KoreanTextDataset(dataset, word2idx)

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 데이터 확인
for batch in train_loader:
    inputs, texts = batch
    print(f"입력 텐서: {inputs.shape}")  # (32, 20)
    print(f"샘플 텍스트: {texts[:2]}")
    break


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2295692657.py, line 1)

In [ ]:
#cnn_model.py
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_filters=100, filter_sizes=[2,3,4]):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embed_dim)) for fs in filter_sizes
        ])
        self.fc = nn.Linear(num_filters * len(filter_sizes), vocab_size)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        conv_x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled_x = [F.max_pool1d(c, c.size(2)).squeeze(2) for c in conv_x]
        out = torch.cat(pooled_x, dim=1)
        return self.fc(out)


In [ ]:
#train.py
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from load_dataset import train_dataset, word2idx  # 데이터셋 불러오기
from cnn_model import TextCNN  # CNN 모델 불러오기

# GPU 또는 CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 경로 설정
MODEL_PATH = "trained_model.pth"

# 기존 모델이 있다면 불러오기 (Fine-tuning 지원)
if os.path.exists(MODEL_PATH):
    print("🔄 기존 모델을 불러와 추가 학습을 진행합니다...")
    model = TextCNN(len(word2idx)).to(device)
    model.load_state_dict(torch.load(MODEL_PATH))
else:
    print("🆕 새로운 모델을 생성하여 학습을 시작합니다...")
    model = TextCNN(len(word2idx)).to(device)

# 옵티마이저 및 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 학습 루프
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs, _ = batch
        inputs = inputs.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        targets = inputs[:, 1:]  # Shifted target (예제)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# 학습 완료 후 모델 저장
torch.save(model.state_dict(), MODEL_PATH)
print(f"✅ 학습 완료! 모델이 '{MODEL_PATH}' 파일로 저장되었습니다.")


In [ ]:
#test_model.py
import os
import torch
from eunjeon import Mecab
from load_dataset import word2idx, idx2word  # 기존 데이터셋에서 불러옴
from cnn_model import TextCNN  # CNN 모델 로드

# 모델 경로 설정
MODEL_PATH = "trained_model.pth"

# GPU 또는 CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 불러오기
if os.path.exists(MODEL_PATH):
    print("📥 저장된 모델을 불러옵니다...")
    model = TextCNN(len(word2idx)).to(device)
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
else:
    print("❌ 저장된 모델이 없습니다. 먼저 train.py를 실행하여 학습해주세요.")
    exit()

# 형태소 분석기 초기화
mecab = Mecab()

def generate_text(model, word2idx, input_text, max_len=20):
    tokens = mecab.morphs(input_text)
    input_idx = [word2idx.get(tok, 0) for tok in tokens]

    # 패딩 적용
    if len(input_idx) < max_len:
        input_idx += [0] * (max_len - len(input_idx))
    else:
        input_idx = input_idx[:max_len]

    input_tensor = torch.tensor([input_idx]).to(device)

    with torch.no_grad():
        output = model(input_tensor)

    predicted_idx = torch.argmax(output, dim=-1).cpu().numpy()
    predicted_text = [idx2word.get(idx, "<UNK>") for idx in predicted_idx[0]]

    return " ".join(predicted_text)

# 사용자 입력을 받아서 문장 생성
while True:
    user_input = input("\n💬 문장을 입력하세요 (종료: exit): ")
    if user_input.lower() == "exit":
        print("🔚 종료합니다.")
        break

    generated_sentence = generate_text(model, word2idx, user_input)
    print(f"📝 생성된 문장: {generated_sentence}")

